In [108]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import vincenty

foursquare_data = pd.read_csv("./data/foursquare-nyc-and-tokyo-check-ins/dataset_TSMC2014_NYC.csv")
climate_data = pd.read_csv("./data/1087533.csv")

/Users/frankyliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,10,11,14,15,20,23,25,40,68,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [109]:
pd.options.display.max_columns = 999
climate_data.HOURLYPrecip = climate_data.HOURLYPrecip.fillna(0.00).replace('T', '0.00').str.replace(r"[a-zA-Z]",'').astype(float)

In [73]:
foursquare_data.columns

Index(['userId', 'venueId', 'venueCategoryId', 'venueCategory', 'latitude',
       'longitude', 'timezoneOffset', 'utcTimestamp'],
      dtype='object')

In [110]:
def closest_station(point, points):
    closest_station = ""
    closest_distance = 50000                                                 
    for station in points:
        cur_dist = vincenty(point, station).kilometers
        if cur_dist < closest_distance:
            closest_station = station_lat_long[station_lat_long['lat_long'] == station]['STATION_NAME'].values[0]
    return closest_station

station_lat_long = climate_data[['STATION_NAME', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
foursquare_data['lat_long'] = [(x, y) for x,y in zip(foursquare_data['latitude'], foursquare_data['longitude'])]                                       
station_lat_long['lat_long'] = [(x, y) for x,y in zip(station_lat_long['LATITUDE'], station_lat_long['LONGITUDE'])]

foursquare_data['closestStation'] = [closest_station(pt, station_lat_long['lat_long']) for pt in foursquare_data['lat_long']]



In [111]:
climate_data.DATE = pd.to_datetime(climate_data.DATE, utc=True)
climate_data.sort_values('DATE', inplace=True)
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp, utc=True)
foursquare_data.sort_values('utcTimestamp', inplace=True)

In [6]:
climate_data.DATE.dt.round('5min')
foursquare_data.utcTimestamp = pd.to_datetime(foursquare_data.utcTimestamp).dt.round('5min')

0       2012-04-12 00:50:00
1       2012-04-12 01:00:00
2       2012-04-12 01:50:00
3       2012-04-12 02:50:00
4       2012-04-12 03:50:00
5       2012-04-12 04:00:00
6       2012-04-12 04:50:00
7       2012-04-12 05:50:00
8       2012-04-12 06:50:00
9       2012-04-12 07:00:00
10      2012-04-12 07:50:00
11      2012-04-12 08:50:00
12      2012-04-12 09:50:00
13      2012-04-12 10:00:00
14      2012-04-12 10:50:00
15      2012-04-12 11:50:00
16      2012-04-12 12:50:00
17      2012-04-12 13:00:00
18      2012-04-12 13:50:00
19      2012-04-12 14:50:00
20      2012-04-12 15:50:00
21      2012-04-12 16:00:00
22      2012-04-12 16:50:00
23      2012-04-12 17:50:00
24      2012-04-12 18:50:00
25      2012-04-12 19:00:00
26      2012-04-12 19:50:00
27      2012-04-12 20:50:00
28      2012-04-12 21:50:00
29      2012-04-12 22:00:00
                ...        
33659   2013-02-16 04:50:00
33660   2013-02-16 05:50:00
33661   2013-02-16 05:50:00
33662   2013-02-16 06:50:00
33663   2013-02-16 0

In [112]:
climate_data['roundedDate'] = climate_data.DATE.dt.round('60min')
foursquare_data['roundedDate'] = foursquare_data.utcTimestamp.dt.round('60min')
foursquare_climate_df = pd.merge(foursquare_data, climate_data,  how='inner', left_on=['closestStation','roundedDate'], right_on = ['STATION_NAME','roundedDate'])

In [113]:
foursquare_climate_df[foursquare_climate_df.venueId == '49c54c2cf964a520ee561fe3']
foursquare_climate_df[foursquare_climate_df.venueId == '3fd66200f964a52021e41ee3']

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,lat_long,closestStation,roundedDate,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
47250,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 21:51:00,FM-15,OVC:08 120,10.00,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.12,1.0,NaN,30.14,0.00,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
47251,375,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-28 21:31:49,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-28 22:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-28 22:00:00,FM-12,NaN,9.94,NaN,53,11.7,41.0,5.3,25,-3.9,34.0,11.0,280,NaN,30.11,1.0,-0.03,30.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
48369,122,3fd66200f964a52021e41ee3,4bf58dd8d48988d116941735,Bar,40.719854,-73.988492,-240,2012-04-29 03:07:26,"(40.71985365, -73.98849235)",JFK INTERNATIONAL AIRPORT NY US,2012-04-29 03:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-29 02:51:00,FM-15,NaN,10.00,NaN,47,8.3,40.0,4.2,29,-1.7,50.0,3.0,330,NaN,30.10,NaN,NaN,30.12,0.00,30.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455,1850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [114]:
foursquare_climate_df.to_csv("foursquare_climate.csv", sep='\t')

In [116]:
foursquare_climate_df[foursquare_climate_df.userId == 878]

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,lat_long,closestStation,roundedDate,STATION,STATION_NAME,ELEVATION,LATITUDE,LONGITUDE,DATE,REPORTTPYE,HOURLYSKYCONDITIONS,HOURLYVISIBILITY,HOURLYPRSENTWEATHERTYPE,HOURLYDRYBULBTEMPF,HOURLYDRYBULBTEMPC,HOURLYWETBULBTEMPF,HOURLYWETBULBTEMPC,HOURLYDewPointTempF,HOURLYDewPointTempC,HOURLYRelativeHumidity,HOURLYWindSpeed,HOURLYWindDirection,HOURLYWindGustSpeed,HOURLYStationPressure,HOURLYPressureTendency,HOURLYPressureChange,HOURLYSeaLevelPressure,HOURLYPrecip,HOURLYAltimeterSetting,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYDeptFromNormalAverageTemp,DAILYAverageRelativeHumidity,DAILYAverageDewPointTemp,DAILYAverageWetBulbTemp,DAILYHeatingDegreeDays,DAILYCoolingDegreeDays,DAILYSunrise,DAILYSunset,DAILYWeather,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageStationPressure,DAILYAverageSeaLevelPressure,DAILYAverageWindSpeed,DAILYPeakWindSpeed,PeakWindDirection,DAILYSustainedWindSpeed,DAILYSustainedWindDirection,MonthlyMaximumTemp,MonthlyMinimumTemp,MonthlyMeanTemp,MonthlyAverageRH,MonthlyDewpointTemp,MonthlyWetBulbTemp,MonthlyAvgHeatingDegreeDays,MonthlyAvgCoolingDegreeDays,MonthlyStationPressure,MonthlySeaLevelPressure,MonthlyAverageWindSpeed,MonthlyTotalSnowfall,MonthlyDeptFromNormalMaximumTemp,MonthlyDeptFromNormalMinimumTemp,MonthlyDeptFromNormalAverageTemp,MonthlyDeptFromNormalPrecip,MonthlyTotalLiquidPrecip,MonthlyGreatestPrecip,MonthlyGreatestPrecipDate,MonthlyGreatestSnowfall,MonthlyGreatestSnowfallDate,MonthlyGreatestSnowDepth,MonthlyGreatestSnowDepthDate,MonthlyDaysWithGT90Temp,MonthlyDaysWithLT32Temp,MonthlyDaysWithGT32Temp,MonthlyDaysWithLT0Temp,MonthlyDaysWithGT001Precip,MonthlyDaysWithGT010Precip,MonthlyDaysWithGT1Snow,MonthlyMaxSeaLevelPressureValue,MonthlyMaxSeaLevelPressureDate,MonthlyMaxSeaLevelPressureTime,MonthlyMinSeaLevelPressureValue,MonthlyMinSeaLevelPressureDate,MonthlyMinSeaLevelPressureTime,MonthlyTotalHeatingDegreeDays,MonthlyTotalCoolingDegreeDays,MonthlyDeptFromNormalHeatingDD,MonthlyDeptFromNormalCoolingDD,MonthlyTotalSeasonToDateHeatingDD,MonthlyTotalSeasonToDateCoolingDD
973,878,4de797941fc7fcfe04ac1dcf,4bf58dd8d48988d12d951735,Ferry,40.731444,-73.963239,-240,2012-04-12 12:35:53,"(40.73144384, -73.96323917)",JFK INTERNATIONAL AIRPORT NY US,2012-04-12 13:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-12 12:51:00,FM-15,BKN:07 60 BKN:07 110 BKN:07 270,10.00,NaN,62,16.7,47.0,8.1,27,-2.8,26.0,15.0,010,23.0,29.97,6.0,NaN,29.99,0.00,29.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520,1832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
974,878,4de797941fc7fcfe04ac1dcf,4bf58dd8d48988d12d951735,Ferry,40.731444,-73.963239,-240,2012-04-12 12:35:53,"(40.73144384, -73.96323917)",JFK INTERNATIONAL AIRPORT NY US,2012-04-12 13:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-12 13:00:00,FM-12,NaN,9.94,NaN,62,16.7,47.0,8.1,27,-2.8,26.0,15.0,010,NaN,29.96,6.0,+0.02,29.99,NaN,NaN,NaN,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520,1832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9999,-9999,NaN,-9999,-9999,NaN,NaN,NaN,NaN,NaN,NaN
1325,878,49fa353af964a520da6d1fe3,4bf58dd8d48988d175941735,Gym / Fitness Center,40.745186,-73.976899,-240,2012-04-12 13:46:24,"(40.74518568, -73.97689926)",JFK INTERNATIONAL AIRPORT NY US,2012-04-12 14:00:00,WBAN:94789,JFK INTERNATIONAL AIRPORT NY US,3.4,40.6386,-73.7622,2012-04-12 13:51:00,FM-15,BKN:07 70 BKN:07 85 BKN:07 120,10.00,NaN,58,14.4,44.0,6.5,24,-4.4,27.0,17.0,350,26.0,29.98,NaN,NaN,30.00,0.00,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,520,1832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN